In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle

# Import Feature Engineered Sales Transaction file
df = pd.read_csv('olist_processado.csv')

## Construir Matriz de Correlação para as relações Cliente-Produto (usando recomendação baseada em Usuário-Usuário) 

In [2]:
df.head(2)

,customer_id,customer_city,customer_state,order_id,order_purchase_timestamp,order_item_id,product_id,price,product_category_name,customer_lat,customer_long
0,06b8999e2fba1a1fbc88172c00ba8bc7,franca,SP,00e7ee1b050b8499577073aeb2a297a1,2017-05-16 15:05:35,1,a9516a079e37a9c9c36b9b78b10169e8,124.99,moveis_escritorio,-20.500253,-47.400367
1,8912fc0c3bbf1e2fbf35819e21706718,santarem,PA,c1d2b34febe9cd269e378117d6681172,2017-11-09 00:50:13,1,a9516a079e37a9c9c36b9b78b10169e8,112.99,moveis_escritorio,-2.441535,-54.720801


In [22]:
print(df.groupby(['product_category_name','customer_id']).agg({'product_id':'count'}))
#df['qtd'] = df.groupby(['product_id']).size()

                                                            product_id
product_category_name     customer_id                                 
agro_industria_e_comercio 0074a15febc9459ab2f999ea0e22c8d6           1
                          01122215dd21ac872ae567ec4e351e01           1
                          01c79a7d3c3d25b655514206df838701           1
                          04d5cb41e92cebd0f1051d38c10f5ec4           1
                          05455dfa7cd02f13d132aa7a6a9729c6           6
...                                                                ...
utilidades_domesticas     ffaded93e5a1fce06363cfb6905676a1           1
                          ffb0327dd973137e6795f0a714f4aa0e           1
                          ffb9d77106a6c70e8853d213c7ad8c6f           1
                          ffba8d414efad51ed575616d51e326b5           1
                          ffc2b93107257d7387cab8801eb10042           1

[98019 rows x 1 columns]


In [34]:
# Encontre a quantidade total adquirida por cada cliente de cada produto
prod_cliente_qty_df = df.groupby(['product_category_name','customer_id']).agg({'product_id':'count'})

# Reseta o index convertendo em colunas
prod_cliente_qty_df.reset_index(inplace=True)


# Encontre o número de clientes únicos que compraram cada produto
prod_cliente_count_df = df.groupby(['product_category_name']).agg({'customer_id':'nunique'})

# Defina a coluna de contagem de clientes
prod_cliente_count_df.columns=['no_clientes']

# Reseta o index convertendo em colunas
prod_cliente_count_df.reset_index(inplace=True)


# Merge a contagem exclusiva do cliente e a quantidade adquirida de cada produto
prod_cliente_df = pd.merge(prod_cliente_qty_df, prod_cliente_count_df, how='inner',on='product_category_name')


# Crie uma tabela dinâmica com todos os produtos em colunas e clientes em linhas e preço como valores
prod_cliente_pivot_df = prod_cliente_df.pivot(index='customer_id',columns='product_category_name',values='product_id').fillna(0)

# Encontre a correlação entre cada dois clientes e construa uma matriz de correlação usando o método corr ()
# Método de Spearman usado para identificar a correlação. Pearson não estava fornecendo melhores resultados e Kendall está demorando muito para ser executado.
cliente_correlacao_df_p = prod_cliente_pivot_df.corr(method='spearman',min_periods=5)
cliente_correlacao_df

product_category_name,agro_industria_e_comercio,alimentos,alimentos_bebidas,artes,artes_e_artesanato,artigos_de_festas,artigos_de_natal,audio,automotivo,bebes,...,pet_shop,portateis_casa_forno_e_cafe,portateis_cozinha_e_preparadores_de_alimentos,relogios_presentes,seguros_e_servicos,sinalizacao_e_seguranca,tablets_impressao_imagem,telefonia,telefonia_fixa,utilidades_domesticas
product_category_name,,,,,,,,,,,,,,,,,,,,,
agro_industria_e_comercio,1.000000,-0.002952,-0.002094,-0.001975,-0.000666,-0.000867,-0.001572,-0.002602,-0.008842,-0.007568,...,-0.005791,-0.001203,-0.000519,-0.010718,-0.000196,-0.001644,-0.001234,-0.009193,-0.002047,-0.010978
alimentos,-0.002952,1.000000,-0.003297,-0.003110,-0.001048,-0.001365,-0.002475,-0.004097,-0.013923,-0.011916,...,-0.007970,-0.001894,-0.000818,-0.016877,-0.000309,-0.002588,-0.001944,-0.014475,-0.000006,-0.017286
alimentos_bebidas,-0.002094,-0.003297,1.000000,-0.002206,-0.000744,-0.000969,-0.001755,-0.002906,-0.009877,-0.008454,...,-0.006469,-0.001343,-0.000580,-0.011973,-0.000219,-0.001836,-0.001379,-0.010269,-0.002287,-0.010448
artes,-0.001975,-0.003110,-0.002206,1.000000,-0.000702,-0.000914,-0.001656,-0.002741,-0.009316,-0.007974,...,-0.006102,-0.001267,-0.000547,-0.011293,-0.000207,-0.001732,-0.001300,-0.009686,-0.002157,-0.011567
artes_e_artesanato,-0.000666,-0.001048,-0.000744,-0.000702,1.000000,-0.000308,-0.000558,-0.000924,-0.003141,-0.002688,...,-0.002057,-0.000427,-0.000185,-0.003807,-0.000070,-0.000584,-0.000438,-0.003265,-0.000727,-0.003899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sinalizacao_e_seguranca,-0.001644,-0.002588,-0.001836,-0.001732,-0.000584,-0.000760,-0.001378,-0.002281,-0.007753,-0.005039,...,-0.005078,-0.001055,-0.000455,-0.009399,-0.000172,1.000000,-0.001082,-0.008061,-0.001795,-0.009627
tablets_impressao_imagem,-0.001234,-0.001944,-0.001379,-0.001300,-0.000438,-0.000571,-0.001035,-0.001713,-0.005822,-0.004983,...,-0.003813,-0.000792,-0.000342,-0.007058,-0.000129,-0.001082,1.000000,-0.006053,-0.001348,-0.007229
telefonia,-0.009193,-0.014475,-0.010269,-0.009686,-0.003265,-0.004252,-0.007707,-0.012759,-0.042596,-0.037115,...,-0.028401,-0.005898,-0.002547,-0.052149,-0.000963,-0.008061,-0.006053,1.000000,-0.009007,-0.053840


### Salva o Cliente para a Matriz de Correlação do Cliente em um arquivo .csv

In [25]:
cliente_correlacao_df.to_csv('cliente-cliente-matriz-correlacao.csv')

### Cria um arquivo Pickle (.pkl) com o dataframe da Matriz de Correlação

In [26]:
pickle.dump(cliente_correlacao_df, open('cliente_correlacao_modelo.pkl','wb'))

In [27]:
cliente_correlacao_df_p = prod_cliente_pivot_df.corr(method='pearson',min_periods=5)

In [28]:
cliente_correlacao_df_p

product_category_name,agro_industria_e_comercio,alimentos,alimentos_bebidas,artes,artes_e_artesanato,artigos_de_festas,artigos_de_natal,audio,automotivo,bebes,...,pet_shop,portateis_casa_forno_e_cafe,portateis_cozinha_e_preparadores_de_alimentos,relogios_presentes,seguros_e_servicos,sinalizacao_e_seguranca,tablets_impressao_imagem,telefonia,telefonia_fixa,utilidades_domesticas
product_category_name,,,,,,,,,,,,,,,,,,,,,
agro_industria_e_comercio,1.000000,-0.002372,-0.001615,-0.001641,-0.000576,-0.000737,-0.001186,-0.002247,-0.007114,-0.006280,...,-0.004714,-0.001053,-0.000445,-0.008897,-0.000173,-0.001195,-0.001065,-0.007482,-0.001494,-0.008324
alimentos,-0.002372,1.000000,-0.002633,-0.002676,-0.000939,-0.001201,-0.001934,-0.003663,-0.011599,-0.010239,...,-0.006179,-0.001717,-0.000725,-0.014506,-0.000282,-0.001949,-0.001736,-0.012199,-0.000676,-0.013571
alimentos_bebidas,-0.001615,-0.002633,1.000000,-0.001822,-0.000639,-0.000818,-0.001317,-0.002494,-0.007896,-0.006971,...,-0.005232,-0.001169,-0.000494,-0.009875,-0.000192,-0.001327,-0.001182,-0.008305,-0.001658,-0.007846
artes,-0.001641,-0.002676,-0.001822,1.000000,-0.000649,-0.000831,-0.001338,-0.002534,-0.008025,-0.007084,...,-0.005317,-0.001188,-0.000502,-0.010036,-0.000195,-0.001348,-0.001201,-0.008440,-0.001685,-0.009389
artes_e_artesanato,-0.000576,-0.000939,-0.000639,-0.000649,1.000000,-0.000291,-0.000469,-0.000889,-0.002815,-0.002485,...,-0.001865,-0.000417,-0.000176,-0.003520,-0.000068,-0.000473,-0.000421,-0.002961,-0.000591,-0.003294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sinalizacao_e_seguranca,-0.001195,-0.001949,-0.001327,-0.001348,-0.000473,-0.000605,-0.000975,-0.001846,-0.005844,-0.004336,...,-0.003872,-0.000865,-0.000366,-0.007309,-0.000142,1.000000,-0.000874,-0.006147,-0.001227,-0.006838
tablets_impressao_imagem,-0.001065,-0.001736,-0.001182,-0.001201,-0.000421,-0.000539,-0.000868,-0.001644,-0.005205,-0.004595,...,-0.003449,-0.000770,-0.000326,-0.006509,-0.000127,-0.000874,1.000000,-0.005474,-0.001093,-0.006090
telefonia,-0.007482,-0.012199,-0.008305,-0.008440,-0.002961,-0.003788,-0.006101,-0.011553,-0.035475,-0.032297,...,-0.024240,-0.005414,-0.002288,-0.045427,-0.000889,-0.006147,-0.005474,1.000000,-0.007058,-0.042806


In [29]:
cliente_correlacao_df_k = prod_cliente_pivot_df.corr(method='kendall',min_periods=5)

In [30]:
cliente_correlacao_df_k

product_category_name,agro_industria_e_comercio,alimentos,alimentos_bebidas,artes,artes_e_artesanato,artigos_de_festas,artigos_de_natal,audio,automotivo,bebes,...,pet_shop,portateis_casa_forno_e_cafe,portateis_cozinha_e_preparadores_de_alimentos,relogios_presentes,seguros_e_servicos,sinalizacao_e_seguranca,tablets_impressao_imagem,telefonia,telefonia_fixa,utilidades_domesticas
product_category_name,,,,,,,,,,,,,,,,,,,,,
agro_industria_e_comercio,1.000000,-0.002951,-0.002093,-0.001975,-0.000666,-0.000867,-0.001571,-0.002601,-0.008833,-0.007563,...,-0.005786,-0.001203,-0.000519,-0.010709,-0.000196,-0.001643,-0.001234,-0.009183,-0.002047,-0.010948
alimentos,-0.002951,1.000000,-0.003296,-0.003109,-0.001048,-0.001365,-0.002474,-0.004095,-0.013907,-0.011908,...,-0.007941,-0.001893,-0.000818,-0.016861,-0.000309,-0.002587,-0.001943,-0.014457,-0.000014,-0.017237
alimentos_bebidas,-0.002093,-0.003296,1.000000,-0.002206,-0.000744,-0.000968,-0.001755,-0.002906,-0.009866,-0.008448,...,-0.006463,-0.001343,-0.000580,-0.011962,-0.000219,-0.001836,-0.001379,-0.010257,-0.002286,-0.010408
artes,-0.001975,-0.003109,-0.002206,1.000000,-0.000701,-0.000914,-0.001656,-0.002741,-0.009307,-0.007969,...,-0.006097,-0.001267,-0.000547,-0.011284,-0.000207,-0.001732,-0.001300,-0.009676,-0.002157,-0.011536
artes_e_artesanato,-0.000666,-0.001048,-0.000744,-0.000701,1.000000,-0.000308,-0.000558,-0.000924,-0.003138,-0.002687,...,-0.002055,-0.000427,-0.000184,-0.003804,-0.000070,-0.000584,-0.000438,-0.003262,-0.000727,-0.003889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sinalizacao_e_seguranca,-0.001643,-0.002587,-0.001836,-0.001732,-0.000584,-0.000760,-0.001378,-0.002281,-0.007745,-0.005037,...,-0.005073,-0.001054,-0.000455,-0.009390,-0.000172,1.000000,-0.001082,-0.008051,-0.001795,-0.009599
tablets_impressao_imagem,-0.001234,-0.001943,-0.001379,-0.001300,-0.000438,-0.000571,-0.001035,-0.001713,-0.005817,-0.004981,...,-0.003810,-0.000792,-0.000342,-0.007052,-0.000129,-0.001082,1.000000,-0.006047,-0.001348,-0.007210
telefonia,-0.009183,-0.014457,-0.010257,-0.009676,-0.003262,-0.004248,-0.007699,-0.012745,-0.042489,-0.037058,...,-0.028349,-0.005892,-0.002545,-0.052041,-0.000962,-0.008051,-0.006047,1.000000,-0.008961,-0.053641


In [31]:
prod_cliente_qty_df.head()

,product_category_name,customer_id,product_id
0,agro_industria_e_comercio,0074a15febc9459ab2f999ea0e22c8d6,1
1,agro_industria_e_comercio,01122215dd21ac872ae567ec4e351e01,1
2,agro_industria_e_comercio,01c79a7d3c3d25b655514206df838701,1
3,agro_industria_e_comercio,04d5cb41e92cebd0f1051d38c10f5ec4,1
4,agro_industria_e_comercio,05455dfa7cd02f13d132aa7a6a9729c6,6


In [32]:
prod_cliente_count_df.head()

,product_category_name,no_clientes
0,agro_industria_e_comercio,182
1,alimentos,450
2,alimentos_bebidas,227
3,artes,202
4,artes_e_artesanato,23
